In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import mysql.connector
import time
from colorama import Fore, Style

# Headers sözlüğü
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
}

def get_data(url):
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.content, "lxml")
    return soup

def read_urls():
    with open(r'C:\Users\USER\Desktop\veri_manipulasyonu\url_oku.txt') as f:
        urls = f.readlines()
    return [url.strip() for url in urls]

def connect_db():
    # Veritabanı bağlantısı kur
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Ayberk2003",  # Şifre burada belirtildi
        database="program"
    )
    return mydb

def insert_value(mydb, data):
    mycursor = mydb.cursor()
    sql = "INSERT INTO products (title, price, time) VALUES (%s, %s, %s)"
    for value in data:
        val = (value['title'], value['price'], value['time'])
        mycursor.execute(sql, val)
    mydb.commit()
    print(mycursor.rowcount, "record(s) inserted.")

def parse(urllist):
    product_list = []
    for ul in urllist:
        if 'price:' in ul:
            url = ul.split("price:")[0].strip()
            check_price = ul.split("price:")[1].strip()
            
            now = datetime.now()
            current_time = now.strftime("%Y-%m-%d %H:%M:%S")
            soup = get_data(url)
            
            title_tag = soup.find("h1", attrs={"id": "title"})
            title = title_tag.text.strip() if title_tag else "Title not found"
            
            price_tag = soup.find("span", attrs={"class": "a-offscreen"})
            price = price_tag.text.strip().replace("TL", "").replace(".", "").replace(",", ".") if price_tag else None
            
            print("Title: ", title)
            print("Price: ", price)
            print("Check_price: ", check_price)
            if price is not None:
                try:
                    # Fiyatı doğru formata dönüştür
                    price_float = float(price.replace(".", "").replace(",", "."))
                    check_price_float = float(check_price.replace(".", "").replace(",", "."))
                    if check_price_float > price_float:
                        print(Fore.GREEN + "İndirim var!" + Style.RESET_ALL + "\n")
                    else:
                        print("İndirim yok.\n")
                except ValueError:
                    print("Fiyat dönüştürme hatası: Price veya check_price değerleri sayıya dönüştürülemedi.")
            
            info = {"title": title, "price": price_float, "time": current_time}
            product_list.append(info)
    
    return product_list

# Döngüyle otomatik fiyat kontrolü
def automatic_price_check(interval):
    mydb = connect_db()
    urllist = read_urls()
    while True:
        print(f"{datetime.now()} - Fiyat kontrolü başlatılıyor...")
        product_list = parse(urllist)
        
        # Fiyatları veritabanına ekle
        insert_value(mydb, product_list)
        
        # Sonuçları yazdır
        for product in product_list:
            print(product)
        
        # Belirli bir süre bekle ve tekrar başlat
        print(f"Sonraki kontrol {interval/60} dakika sonra yapılacak.")
        time.sleep(interval)

# 30 dakika aralıkla fiyatları kontrol et
automatic_price_check(1800)  # 1800 saniye = 30 dakika


Title:  Apple 2020 MacBook Air Laptop: Apple M1 Çip, 13 inç Retina Ekran, 8 GB RAM, 256 GB SSD Depolama, Arkadan Aydınlatmalı Klavye, FaceTime HD Kamera, Touch ID. iPhone/iPad ile çalışır; Gümüş Rengi
Price:  30299.00
Check_price:  40.000,00
Title:  Apple 2024 M3 çipli 13 inç MacBook Air Laptop: 13.6 inç Liquid Retina Ekran, 8 GB Birleşik Bellek, 512 GB SSD Depolama, 1080p FaceTime HD Kamera, Touch ID. Gece Yarısı
Price:  52239.00
Check_price:  45.000,00
1 record(s) inserted.
[{'title': 'Apple 2020 MacBook Air Laptop: Apple M1 Çip, 13 inç Retina Ekran, 8 GB RAM, 256 GB SSD Depolama, Arkadan Aydınlatmalı Klavye, FaceTime HD Kamera, Touch ID. iPhone/iPad ile çalışır; Gümüş Rengi', 'price': 3029900.0, 'time': '2024-10-08 18:15:21'}, {'title': 'Apple 2024 M3 çipli 13 inç MacBook Air Laptop: 13.6 inç Liquid Retina Ekran, 8 GB Birleşik Bellek, 512 GB SSD Depolama, 1080p FaceTime HD Kamera, Touch ID. Gece Yarısı', 'price': 5223900.0, 'time': '2024-10-08 18:15:23'}]
